# Profilers  

27.4. The Python Profilers  

https://docs.python.org/3/library/profile.html

**Source Codes**

https://hg.python.org/cpython/file/3.5/Lib/profile.py

https://hg.python.org/cpython/file/3.5/Lib/pstats.py

## 27.4.1. Introduction to the profilers

**A profile is a set of statistics** 

that describes **how often** and for **how long** various parts of the program executed.

These statistics can be formatted into reports via the **pstats** module.

The Python standard library provides **two** different implementations of the same profiling interface:

**cProfile** and **profile** provide deterministic profiling of Python programs. 

* **cProfile** is recommended for most users

  it’s a C extension with reasonable overhead that makes it suitable for profiling long-running programs. 
  
  Based on `lsprof`, contributed by Brett Rosen and Ted Czotter.


* **profile**, a pure Python module whose interface is imitated by cProfile, 

   but which adds significant overhead to profiled programs.
   
   If you’re trying to extend the profiler in some way, the task might be easier with this module. 
   
   Originally designed and written by Jim Roskind.

<b>Profilers: profile, cProfile, and pstats</b> – Performance analysis of Python programs.

## 27.4.2. Instant User’s Manual

This section is provided for users that “don’t want to read the manual.” 

It provides a very brief overview, and allows a user to rapidly perform profiling on an existing application.

The most basic starting point in the profile module is 

**run()**

It takes <b>a string statement</b> as argument, 

and creates a **report** of the time spent executing different lines of code while running the statement.   

To profile a function

`re.compile()`

that takes a single argument

In [ ]:
import cProfile
import re
cProfile.run('re.compile("foo|bar")')

### re — Regular expression operations**
   
  https://docs.python.org/3.5/library/re.html
  
  This module provides regular expression matching operations 
  
  **Regular Expression Syntax**

   A regular expression (or RE) specifies **a set of strings** that matches it;
   
   the functions in this module let you check if a particular string matches a given regular expression
   
   or if a given regular expression matches a particular string


** '|' ** 

A|B, where A and B can be arbitrary REs, creates a regular expression that will match either A or B



In [ ]:
import re
m = re.search("foo|bar", 'abcfoo')
m.group(0)

* **re.compile(pattern, flags=0)**

    Compile a regular expression pattern into a regular expression object,
    
    which can be used for matching using its match() and search() methods,


In [ ]:
pattern = re.compile("foo|bar")
m1=pattern.search("foo")  
m2=pattern.search("bar")
print(m1.group(0))
print(m2.group(0))

In [ ]:
import cProfile
import re
cProfile.run('re.compile("foo|bar")')

* **The first line**
   
   indicates that 197 calls were monitored.
   
   Of those calls, 192 were primitive, meaning that the call was not induced via recursion. 


* **The next line**

   Ordered by: standard name, 
   
   indicates that the text string in the far right column was used to sort the output. 


* ** The column headings include:**

   * <b>ncalls</b>: the number of calls,
   
     When there are two numbers in the the column 
   
     **3/1**
     
     it means that the function recursed.
     
     The second value is the number of primitive calls and the former is the total number of calls. 
     
     Note that when the function does not recurse, these two values are the same, and only the single figure is printed

   
   * <b>tottime</b>: the total time spent in the given function (and excluding time made in calls to sub-functions)

   * <b>percall</b> is the quotient of tottime divided by ncalls

   * <b>cumtime</b> is the cumulative time spent in this and all <b>subfunctions</b> (from invocation till exit). 
  
   * <b>percall</b> is the quotient of cumtime divided by primitive calls

   * <b>filename:lineno(function)</b> provides the respective data of each function

### The results to a file
  
<b>1 Save the results to a file</b>

by specifying a filename to the `run()` function:

In [ ]:
import cProfile
import re
cProfile.run('re.compile("foo|bar")', 'restats')

2 **manipulating and printing the data** saved into a profile results file

* **The `pstats` module’s `Stats` class **

    a variety of methods for manipulating and printing the data saved into a profile results file:

In [ ]:
import pstats
p = pstats.Stats('restats')
p.strip_dirs().sort_stats(-1).print_stats()

* **strip_dirs()**：　removed the extraneous path from all the module names. 

* **sort_stats()**：　sorted all the entries according to the standard 
  　　
   <b>module/line/name</b>
    
   string that is printed.

* **print_stats()**： method printed out all the statistics.

#### You might try the following sort calls:

In [ ]:
p.sort_stats('name')
p.print_stats()

The first call will actually sort the list by function name,

The second call will print out the statistics. 

#### The following are some interesting calls to experiment with:

1  understand what <b>algorithms are taking time</b>.

This sorts the profile by <b>cumulative time</b> in a function, and then only prints <b>the ten most significant lines</b>. 


In [ ]:
p.sort_stats('cumulative').print_stats(10)

2 looking to see <b>what functions were looping a lot</b>, and taking a lot of time:

to sort according to time spent within each function, and then print the statistics for the top ten functions.

In [ ]:
p.sort_stats('time').print_stats(10)

3 <b>sort all the statistics by file name</b>, 

and then print out statistics for <b>only the class init methods</b> (they are spelled with __init__ in them).

In [ ]:
p.sort_stats('file').print_stats('__init__')

**4** This line sorts statistics with 

 <b>a primary key of time</b>
 
 <b>a secondary key of cumulative time</b>, 

and then prints out some of the statistics. 

To be specific, the list is 

first culled down to <b>50% (re: .5)</b> of its original size, 

then only lines containing <b>init</b> are maintained, 

and that sub-sub-list is printed.

In [ ]:
p.sort_stats('time', 'cumulative').print_stats(.5, 'init')

## 27.4.7. Calibration

The profiler of the <b>profile</b> module <b>subtracts a constant</b> 

from each event handling time to compensate for 

the overhead of calling the time function

By default, the constant is 0.

The following procedure can be used to obtain a better constant for a given platform.


In [ ]:
import profile
pr = profile.Profile()

your_computed_bias=pr.calibrate(10000)

print(your_computed_bias)
    

The method executes the number of Python calls given by the argument, 

<b>directly</b> 

<b>under the profiler</b>

   measuring the time for both. 

It then computes the hidden overhead per profiler event, and returns that as a float. 

The object of this exercise is to get a fairly consistent result. 

If your computer is very fast, or your timer function has poor resolution, you might have to pass 100000, or even 1000000, to get consistent results.

#### When you have a consistent answer, there are <b>three ways</b> you can use it:


In [ ]:
import profile

# 1. Apply computed bias to all Profile instances created hereafter.
profile.Profile.bias = your_computed_bias

# 2. Apply computed bias to a specific Profile instance.
pr = profile.Profile()
pr.bias = your_computed_bias

# 3. Specify computed bias in instance constructor.
pr = profile.Profile(bias=your_computed_bias)

## Example: profiling seuif97

In [25]:
# -*- coding: utf-8 -*-

import seuif97
import cProfile
import pstats
import io

p = 16.10
t = 535.10

pr = cProfile.Profile()
pr.enable()

h = seuif97.pt2h(p, t)
#s = seuif97.pt2s(p, t)

pr.disable()

s = io.StringIO() # 1 n-memory text streams 

sortby = 'cumulative'

ps = pstats.Stats(pr, stream=s).sort_stats(sortby) # 2 Stats in In-memory text streams

ps.print_stats()

print(s.getvalue()) # 3 get In-memory text streams


         20 function calls in 0.000 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 C:\Python35\lib\site-packages\IPython\core\interactiveshell.py:2855(run_code)
        2    0.000    0.000    0.000    0.000 C:\Python35\lib\codeop.py:132(__call__)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        1    0.000    0.000    0.000    0.000 <ipython-input-25-bfd343c57ed2>:14(<module>)
        1    0.000    0.000    0.000    0.000 C:\Python35\lib\seuif97.py:11(pt2h)
        1    0.000    0.000    0.000    0.000 <ipython-input-25-bfd343c57ed2>:17(<module>)
        2    0.000    0.000    0.000    0.000 C:\Python35\lib\site-packages\IPython\core\hooks.py:127(__call__)
        2    0.000    0.000    0.000    0.000 C:\Python35\lib\site-packages\IPython\utils\ipstruct.py:125(

### 6.2. io — Core tools for working with streams

https://docs.python.org/3/library/io.html
    
The io module provides Python’s main facilities for dealing with various types of I/O.

There are three main types of I/O: **text I/O, binary I/O and raw I/O**    

**In-memory text streams** are also available as **StringIO** objects:
   

In [ ]:
f = io.StringIO("some initial text data")
f.getvalue()

## Example: profiling fibonacci
 
This <b>recursive</b> version of a <b>fibonacci</b> sequence calculator

is especially useful for demonstrating the profile 

because we can improve the performance so much.

The standard report format shows a summary and then details for each function executed.
    

In [1]:
import cProfile

def fib(n):
    # https://en.wikipedia.org/wiki/Fibonacci_number
    # http://en.literateprograms.org/Fibonacci_numbers_(Python)
    if n == 0 or n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def fib_seq(n):
    
    seq = [ ]
    if n > 0:
        seq.extend(fib_seq(n-1))
    seq.append(fib(n))
    
    return seq

print('RAW')
print('=' * 80)
cProfile.run('print(fib_seq(20)); print')


RAW
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946]
         57369 function calls (79 primitive calls) in 0.028 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     21/1    0.000    0.000    0.028    0.028 <ipython-input-1-93ba4aebaf67>:11(fib_seq)
 57291/21    0.027    0.000    0.027    0.001 <ipython-input-1-93ba4aebaf67>:3(fib)
        1    0.000    0.000    0.028    0.028 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 iostream.py:227(_is_master_process)
        2    0.000    0.000    0.000    0.000 iostream.py:240(_schedule_flush)
        2    0.000    0.000    0.000    0.000 iostream.py:308(write)
        1    0.000    0.000    0.028    0.028 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        2    0.000 

Not surprisingly, most of the time here is spent calling <b>`fib()` repeatedly </b>. 

<img src="./img/recursion_without_cache.png"/> 


It’s a very inefficient algorithm: 

the amount of function calls increases <b>exponentially</b> for increasing values of <b>n</b>

because the function calls values that it has already calculated again and again.

We needed to speed up a lot of my recursive algorithms.

### ** Decorators**  really came to the rescue in the form of **memoization**

https://en.wikipedia.org/wiki/Memoization

The easy way to optimize this would be to 

* **cache the values in a dictionary** 


* check to see if that value of <b>n</b> has been called previously. 

  * If it has, return it’s value in the dictionary, 

  * if not, proceed to call the function. 
  
This is ** memoization **.


In [10]:
class memoize:
    
    # from http://avinashv.net/2008/04/python-decorators-syntactic-sugar/
    def __init__(self, function):
        self.function = function
       
       #　a dictionary, ｀self.memoized｀, that acts as our cache
        self.memoized = {}

    def __call__(self, *args):
        
        try:
           
           return self.memoized[args]  # 
        
        except KeyError:
            self.memoized[args] = self.function(*args)
            return self.memoized[args]

There is now a dictionary, **self.memoized**, that acts as our cache, 

and a change in the exception handling that looks for ｀KeyError｀, 

which throws an error if a key doesn’t exist in a dictionary. 

Again, this class is generalized, and will work for any recursive function that could benefit from memoization.

### a memoize decorator

We can add <b>a memoize decorator</b> to reduce the number of recursive calls 

and have a big impact on the performance of this function.

In [11]:
import cProfile

@memoize
def fib(n):
    # from http://en.literateprograms.org/Fibonacci_numbers_(Python)
    if n == 0 or n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def fib_seq(n):
    seq = [ ]
    if n > 0:
        seq.extend(fib_seq(n-1))
    seq.append(fib(n))
    return seq

print('MEMOIZED')
print('=' * 80)
cProfile.run('print(fib_seq(20)); print')


MEMOIZED
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946]
         158 function calls (100 primitive calls) in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    59/21    0.000    0.000    0.000    0.000 <ipython-input-10-f0322ac4bf09>:10(__call__)
     21/1    0.000    0.000    0.000    0.000 <ipython-input-11-d04dd80e50cc>:11(fib_seq)
       21    0.000    0.000    0.000    0.000 <ipython-input-11-d04dd80e50cc>:3(fib)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 iostream.py:227(_is_master_process)
        2    0.000    0.000    0.000    0.000 iostream.py:240(_schedule_flush)
        2    0.000    0.000    0.000    0.000 iostream.py:308(write)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
 

By remembering the <b>Fibonacci</b> value at each level we can avoid most of the recursion

the **ncalls** count for **fib()** shows that it never recurses.

###  Decorators

A decorator is any callable Python object that is used to modify a function, method or class definition. 

A decorator is passed the <b>original object</b> being defined and <b>returns a modified object</b>, 



In [25]:
# get square sum
def square_sum(a, b):
    return a**2 + b**2

# get square diff
def square_diff(a, b):
    return a**2 - b**2


print(square_sum(3, 4))
print(square_diff(3, 4))


25
-7


###  modify: print input

In [26]:
# modify: print input

# get square sum
def square_sum(a, b):
    print("intput:", a, b)
    return a**2 + b**2

# get square diff
def square_diff(a, b):
    print("input", a, b)
    return a**2 - b**2


print(square_sum(3, 4))
print(square_diff(3, 4))


intput: 3 4
25
input 3 4
-7


In [27]:
def decorator(func):
   
    def new_func(a, b):
        print("input", a, b)
        return func(a, b)
    
    return new_func

# get square sum
@decorator
def square_sum(a, b):
    return a**2 + b**2

# get square diff
@decorator
def square_diff(a, b):
    return a**2 - b**2

print(square_sum(3, 4))
print(square_diff(3, 4))

input 3 4
25
input 3 4
-7


### See Also： 18.1 Fibonacci Sequences, Revisited

The function `fastFib` has a parameter, `memo`, that it uses to keep track of the numbers it has already evaluated.

In [8]:
#Page 254, Figure 18.3
def fastFib(n, memo = {}):
    """Assumes n is an int >= 0, memo used only by recursive calls
       Returns Fibonacci of n"""
    if n == 0 or n == 1:
        return 1
    try:
        return memo[n]
    except KeyError:
        result = fastFib(n-1, memo) + fastFib(n-2, memo)
        memo[n] = result
        return result


In [9]:
import profile

def fib_seq(n):
    seq = [ ]
    if n > 0:
        seq.extend(fib_seq(n-1))
    seq.append(fastFib(n))
    return seq

print('MEMOIZED')
print('=' * 80)
profile.run('print(fib_seq(20)); print')

MEMOIZED
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946]
         138 function calls (80 primitive calls) in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       21    0.000    0.000    0.000    0.000 :0(append)
        1    0.000    0.000    0.000    0.000 :0(exec)
       20    0.000    0.000    0.000    0.000 :0(extend)
        2    0.000    0.000    0.000    0.000 :0(getpid)
        2    0.000    0.000    0.000    0.000 :0(isinstance)
        1    0.000    0.000    0.000    0.000 :0(print)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        2    0.000    0.000    0.000    0.000 :0(write)
    59/21    0.000    0.000    0.000    0.000 <ipython-input-8-6c19e0a9138f>:2(fastFib)
     21/1    0.000    0.000    0.000    0.000 <ipython-input-9-8368b550d98a>:3(fib_seq)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        2    0.000    0.

## pstats: Saving and Working With Statistics

The standard report created by the profile functions is not very flexible. 

If it doesn’t meet your needs, you can produce your own reports by saving the raw profiling data from `run()` and processing it separately with the `Stats` class from `pstats`.

For example, to run several iterations of the same test and combine the results, you could do something like this:



In [5]:
import profile
import pstats

# from profile_fibonacci_memoized import fib, fib_seq

# Create 5 set of stats
filenames = []
for i in range(5):
    filename ='profile_stats_%d.stats' % i
    profile.run('print("%d " % i,fib_seq(20))', filename)

# Read all 5 stats files into a single object
stats = pstats.Stats('profile_stats_0.stats')
for i in range(1, 5):
    stats.add('profile_stats_%d.stats' % i)

# Clean up filenames for the report
stats.strip_dirs()

# Sort the statistics by the cumulative time spent in the function
stats.sort_stats('cumulative')
stats.print_stats()


0  [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946]
1  [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946]
2  [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946]
3  [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946]
4  [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946]
Mon May 23 21:31:38 2016    profile_stats_0.stats
Mon May 23 21:31:38 2016    profile_stats_1.stats
Mon May 23 21:31:38 2016    profile_stats_2.stats
Mon May 23 21:31:38 2016    profile_stats_3.stats
Mon May 23 21:31:38 2016    profile_stats_4.stats

         565 function calls (465 primitive calls) in 0.000 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       20    0.000    0.000    0.000    0.000 iostream.py:240(_schedule_flush)
        

The output report is sorted in descending order of cumulative time spent in the function and the directory names are
removed from the printed filenames to conserve horizontal space.

### 24.1.4 Limiting Report Contents

Since we are studying the performance of `fib()` and `fib_seq()`, we can also restrict the output report to only
include those functions using a regular expression to match the `filename:lineno(function)` values we want.


In [6]:
import profile
import pstats

#from profile_fibonacci_memoized import fib, fib_seq

# Read all 5 stats files into a single object
stats = pstats.Stats('profile_stats_0.stats')
for i in range(1, 5):
    stats.add('profile_stats_%d.stats' % i)

stats.strip_dirs()
stats.sort_stats('cumulative')

# limit output to lines with "(fib" in them
stats.print_stats('\(fib')


Mon May 23 21:31:38 2016    profile_stats_0.stats
Mon May 23 21:31:38 2016    profile_stats_1.stats
Mon May 23 21:31:38 2016    profile_stats_2.stats
Mon May 23 21:31:38 2016    profile_stats_3.stats
Mon May 23 21:31:38 2016    profile_stats_4.stats

         565 function calls (465 primitive calls) in 0.000 seconds

   Ordered by: cumulative time
   List reduced from 20 to 1 due to restriction <'\\(fib'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    105/5    0.000    0.000    0.000    0.000 <ipython-input-3-d04dd80e50cc>:11(fib_seq)




The regular expression includes a literal left paren (() to match against the function name portion of the location value.

### 24.1.5 Caller / Callee Graphs

Stats also includes methods for printing the callers and callees of functions

In [7]:
import cProfile as profile
import pstats

#from profile_fibonacci_memoized import fib, fib_seq

# Read all 5 stats files into a single object
stats = pstats.Stats('profile_stats_0.stats')
for i in range(1, 5):
    stats.add('profile_stats_%d.stats' % i)

stats.strip_dirs()
stats.sort_stats('cumulative')

print('INCOMING CALLERS:')
stats.print_callers('\(fib')

print('OUTGOING CALLEES:')
stats.print_callees('\(fib')


INCOMING CALLERS:
   Ordered by: cumulative time
   List reduced from 20 to 1 due to restriction <'\\(fib'>

Function                                    was called by...
<ipython-input-3-d04dd80e50cc>:11(fib_seq)  <- <ipython-input-3-d04dd80e50cc>:11(fib_seq)(100)    0.000
                                               <string>:1(<module>)(5)    0.000


OUTGOING CALLEES:
   Ordered by: cumulative time
   List reduced from 20 to 1 due to restriction <'\\(fib'>

Function                                    called...
<ipython-input-3-d04dd80e50cc>:11(fib_seq)  -> :0(append)(105)    0.000
                                               :0(extend)(100)    0.000
                                               <ipython-input-2-760383e4683f>:10(__call__)(105)    0.000
                                               <ipython-input-3-d04dd80e50cc>:11(fib_seq)(100)    0.000


